In [2]:
import torch
import cv2
from pathlib import Path
import os
import logging

logging.basicConfig(filename='process.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
def log_message(message):
    logging.info(message)

In [4]:
import os
# Initialize logging
logging.basicConfig(filename='yolo_training.log', level=logging.INFO)

# Load YOLOv5 model for training (replace 'yolov5s.pt' with your custom model path)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Define directories for training data (Roboflow images)
train_data_dir = Path(r'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/medical_business_data/train/images')
train_labels_dir = Path(r'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/medical_business_data/train/labels')

# Define training parameters
epochs = 100  # Number of epochs
batch_size = 4  # Batch size
img_size = 640  # Image size

# Define the data.yaml file location
data_yaml_path = 'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/medical_business_data/data.yaml'

# Train the model using the YOLOv5 script (run it as a shell command)
# This command runs the actual YOLOv5 training script, passing the necessary arguments
os.system(f'python C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/yolov5/train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {data_yaml_path} --weights yolov5s.pt --cache')

# After training, save the model's weights
torch.save(model.model.state_dict(), 'labeled_yolov5_model.pt')

# Log training completion
logging.info('Training completed and model saved to labeled_yolov5_model.pt')

Using cache found in C:\Users\elbet/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-16 Python-3.9.20 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [15]:
import torch

ckpt = torch.load('C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/notebooks/labeledyolov5.pt', map_location='cpu')
print(ckpt.keys())  # Check if 'model' key exists in the checkpoint


odict_keys(['model.model.0.conv.weight', 'model.model.0.conv.bias', 'model.model.1.conv.weight', 'model.model.1.conv.bias', 'model.model.2.cv1.conv.weight', 'model.model.2.cv1.conv.bias', 'model.model.2.cv2.conv.weight', 'model.model.2.cv2.conv.bias', 'model.model.2.cv3.conv.weight', 'model.model.2.cv3.conv.bias', 'model.model.2.m.0.cv1.conv.weight', 'model.model.2.m.0.cv1.conv.bias', 'model.model.2.m.0.cv2.conv.weight', 'model.model.2.m.0.cv2.conv.bias', 'model.model.3.conv.weight', 'model.model.3.conv.bias', 'model.model.4.cv1.conv.weight', 'model.model.4.cv1.conv.bias', 'model.model.4.cv2.conv.weight', 'model.model.4.cv2.conv.bias', 'model.model.4.cv3.conv.weight', 'model.model.4.cv3.conv.bias', 'model.model.4.m.0.cv1.conv.weight', 'model.model.4.m.0.cv1.conv.bias', 'model.model.4.m.0.cv2.conv.weight', 'model.model.4.m.0.cv2.conv.bias', 'model.model.4.m.1.cv1.conv.weight', 'model.model.4.m.1.cv1.conv.bias', 'model.model.4.m.1.cv2.conv.weight', 'model.model.4.m.1.cv2.conv.bias', 'mod

In [16]:
import torch

ckpt = torch.load('C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/notebooks/yolov5s.pt', map_location='cpu')
print(ckpt.keys())  # Check if 'model' key exists in the checkpoint


dict_keys(['epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'wandb_id', 'date'])


In [17]:
import os
import cv2
from pathlib import Path
import shutil

# Define paths
weights_path = 'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/notebooks/yolov5s.pt'
images_path = 'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/images/'
output_path = 'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/detected_images/'
temp_output_path = 'C:/Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/yolov5/runs/detect/results/'

# Ensure output directories exist
Path(output_path).mkdir(parents=True, exist_ok=True)

# Run detection using detect.py from the YOLOv5 repo
for img_name in os.listdir(images_path):
    img_path = os.path.join(images_path, img_name)
    print(f'Starting object detection for {img_name}.')

    try:
        # Ensure the image is readable
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to read image {img_name}. Skipping...")
            continue

        # Construct the command for detect.py
        command = f'python detect.py --weights {weights_path} --source {img_path} --project {output_path} --name results --exist-ok --conf-thres 0.25 --img-size 640'



        # Run the command
        os.system(command)

        # Define the path of the detected image in the results directory
        detected_img_name = os.path.join(temp_output_path, img_name)

        # Define the final output path for saving the detected image
        final_output_img_path = os.path.join(output_path, f'detected_{img_name}')

        # Move the detected image to the final output directory
        if os.path.exists(detected_img_name):
            shutil.move(detected_img_name, final_output_img_path)
            print(f'Object detection completed and results saved for {img_name} at {final_output_img_path}.')
        else:
            print(f"Detection results not found for {img_name}.")
        
    except Exception as e:
        print(f'Error processing {img_name}: {e}')

print('Completed object detection process for all images.')


Starting object detection for 10.jpg.
Detection results not found for 10.jpg.
Starting object detection for 11.jpg.
Detection results not found for 11.jpg.
Starting object detection for 11891.jpg.
Detection results not found for 11891.jpg.
Starting object detection for 11892.jpg.
Detection results not found for 11892.jpg.
Starting object detection for 11893.jpg.
Detection results not found for 11893.jpg.
Starting object detection for 11894.jpg.
Detection results not found for 11894.jpg.
Starting object detection for 11895.jpg.
Detection results not found for 11895.jpg.
Starting object detection for 11896.jpg.
Detection results not found for 11896.jpg.
Starting object detection for 11897.jpg.
Detection results not found for 11897.jpg.
Starting object detection for 11898.jpg.
Detection results not found for 11898.jpg.
Starting object detection for 11900.jpg.
Detection results not found for 11900.jpg.
Starting object detection for 11901.jpg.
Detection results not found for 11901.jpg.
Star